<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>



<br>

# <font color="#76b900">**Notebook 5:** 處理大型文件</font>

<br>

在上一個 notebook 中，我們學習了 Running State Chain 和知識庫！到最後，我們擁有了進行簡單對話管理和自訂知識追蹤所需的所有工具。在這個 notebook 中，我們將採用相同的想法並轉向大型文件的領域，考慮當我們嘗試將大型檔案納入我們的 LLM 脈絡資訊(Context)時會遇到什麼樣的問題。

**學習目標：**


-   熟悉文件載入器以及它們可能為您提供的實用工具類型。
-   學習如何透過分塊(Chunking)文件並逐步建構知識庫來解析脈絡資訊(Context)空間有限的大型文件。
-   了解文件分塊(document chunks)的漸進式重新脈絡化(progressive recontextualization)、強制(coersion)和整合(consolidation)如何極其有用，以及它會遇到自然限制的地方。

**值得思考的問題：**


-   查看從您的 ArxivParser 出來的分塊，您會注意到有些分塊本身沒有什麼意義，或者在轉換為文字(Text)時已經完全損壞。是否值得對分塊進行清理？
-   考慮文件摘要工作流程（或任何處理大量文件分塊列表的類似工作流程），這應該多久發生一次，什麼時候是合理的？

**環境設置：**


In [1]:
## Necessary for Colab, not necessary for course environment
# %pip install -qq langchain langchain-nvidia-ai-endpoints gradio
# %pip install -qq arxiv pymupdf

# import os
# os.environ["NVIDIA_API_KEY"] = "nvapi-..."

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [2]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
ChatNVIDIA.get_available_models()

/usr/local/lib/python3.11/site-packages/langchain_nvidia_ai_endpoints/_common.py:237: UserWarning: Default model is set as: 01-ai/yi-large. 
Set model using model parameter. 
To get available models use available_models property.
  warnings.warn(


[Model(id='01-ai/yi-large', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-yi-large'], supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='abacusai/dracarys-llama-3.1-70b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='ai21labs/jamba-1.5-large-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='ai21labs/jamba-1.5-mini-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='aisingapore/sea-lion-7b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-sea-lion-7b-instruct'], supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='baichuan-inc/baichu

In [3]:
## Useful utility method for printing intermediate states
from langchain_core.runnables import RunnableLambda
from functools import partial

def RPrint(preface="State: "):
    def print_and_return(x, preface=""):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def PPrint(preface="State: "):
    def print_and_return(x, preface=""):
        pprint(preface, x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))



<br>

## **第一部分：** 與文件聊天


這個 notebook 將開始圍繞使用 LLM 與文件聊天的較長討論串。在一個聊天模型在巨大的公共資料儲存庫上訓練，而在自訂資料上重新訓練它們成本高得令人望而卻步的世界中，讓 LLM 對一組 PDF 甚至 YouTube 影片進行分析推理(Reasoning)的想法開啟了許多機會！

-   **您的 LLM 可以擁有基於人類可讀文件的可修改知識庫，** 這意味著您可以直接控制它有權存取什麼樣的資料，並可以指示它與之互動。
-   **您的 LLM 可以整理並直接從您的文件集中提取參考資料。** 透過充分的提示(Prompt)工程和指令跟隨先驗假設，您可以強制您的模型僅基於您提供的材料行動。
-   **您的 LLM 甚至可能與您的文件互動，根據需要進行自動修改。** 這開啟了自動內容精煉(Refinement)和綜合結果(Synthesis)操作的途徑，這將在稍後探討。

列出一些可能性相當容易，從那裡您可以讓您的想像力自由馳騁...但我們還沒有獲得做這件事的工具，對吧？

<br>

#### **最單純的(Naive)方法：填充(Stuffing)您的文件**

假設您有一些文字(Text)文件（PDF、部落格等）並想詢問與這些文件內容相關的問題。您可以嘗試的一種方法涉及取得文件的表示並將其全部餵給聊天模型！從文件角度來看，這被稱為[**文件填充(document stuffing)**](https://js.langchain.com/v0.1/docs/modules/chains/document/stuff/)。

> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/doc_stuff.png" width=800px/>
>
> 來自 [**Stuff | LangChain**🦜️🔗](https://js.langchain.com/v0.1/docs/modules/chains/document/stuff/)

如果您的模型足夠強大且您的文件足夠短，這可能會運作得很好，但不應該期望它對整個文件運作得很好。許多現代 LLM 由於訓練限制而在處理長脈絡資訊(Context)方面有顯著困難。現在大型模型的惡化並不那麼災難性，但無論您使用哪個模型，良好的指令跟隨可能會很快崩潰（假設您存取的是原始模型）。

<br>


**您需要解決的文件分析推理(Reasoning)的關鍵問題是：**

-   我們如何將文件分割成可以進行分析推理(Reasoning)的片段？
-   隨著文件大小和數量的增加，我們如何有效地找到並考慮這些片段？

這門課程將探索幾種方法來解決這些問題，同時繼續建構我們的 LLM 流程協調管理(Orchestration)技能。***這個 notebook 將擴展我們之前的 Running Chain 技能，用於更漸進的分析推理(Reasoning)制定，而下一個 notebook 將介紹一些新技術來適當地解決大規模檢索(Retrieval)。*** 透過這種體驗，我們將繼續利用尖端的開源解決方案來使我們的解決方案標準化和可整合。

說到這裡，文件載入框架領域有許多強大的選項，兩個主要參與者將在整個課程中出現：

-   [**LangChain**](https://python.langchain.com/docs/get_started/introduction) 透過一般分塊(Chunking)策略和與內嵌框架/服務的強大整合，為將 LLM 連接到您自己的資料來源提供了一個簡單的框架。這個框架最初圍繞其對 LLM 功能的強大一般支援而發展，這表明其積極的優勢更接近鏈(Chain)想法(abstractions)和 Agent 協調。

-   [**LlamaIndex**](https://gpt-index.readthedocs.io/en/stable/) 是一個用於 LLM 應用程式的資料框架，用於輸入(Intake)、結構化和存取私有或特定領域的資料。它後來分支出來包括類似於 LangChain 的一般 LLM 功能，但截至目前，它在解決 LLM 組件的文件方面仍然最強，因為其初始想法(abstractions)圍繞該問題為中心。

建議閱讀更多關於 LlamaIndex 和 LangChain 的獨特優勢，並選擇最適合您的。由於 LlamaIndex 可以*與* LangChain 一起使用，框架的獨特功能可以在沒有太多問題的情況下一起利用。為了簡單起見，我們將在這門課程中堅持使用 LangChain，並將允許 [**NVIDIA/GenerativeAIExamples 儲存庫**](https://github.com/NVIDIA/GenerativeAIExamples/tree/main/RAG/notebooks) 為感興趣的人探索更深入的 LlamaIndex 選項。






<br>

## **第二部分：** 載入文件


LangChain 提供各種[文件載入器](https://python.langchain.com/docs/integrations/document_loaders)來促進從許多不同來源和位置（本地儲存、私有 s3 儲存桶、公共網站、訊息 API 等）輸入(Intake)各種文件格式（HTML、PDF、程式碼）。這些載入器查詢您的資料來源並返回一個 `Document` 物件，該物件包含內容和中介資料(Metadata)，通常採用純文字(Text)或其他人類可讀格式。已經有很多文件載入器構建並準備使用，第一方 LangChain 選項列在[這裡](https://python.langchain.com/docs/integrations/document_loaders)。

**在這個範例中，我們可以使用以下 LangChain 載入器之一載入我們選擇的研究論文：**

-   [`UnstructuredFileLoader`](https://python.langchain.com/docs/integrations/document_loaders/unstructured_file)：適用於任意檔案的一般有用檔案載入器；對您的文件結構不做太多假設，通常是足夠的。

-   [`ArxivLoader`](https://python.langchain.com/docs/integrations/document_loaders/arxiv)：一個更專業的檔案載入器，可以直接與 Arxiv 介面通訊。[只是許多範例中的一個](https://python.langchain.com/docs/integrations/document_loaders)，這將對您的資料做出一些更多假設，以產生更好的解析並自動填充中介資料(Metadata)（當您有多個文件/格式時很有用）。

對於我們的程式碼範例，我們將預設使用 `ArxivLoader` 載入 [MRKL](https://arxiv.org/abs/2205.00445) 或 [ReAct](https://arxiv.org/abs/2210.03629) 發表論文中的一個，因為您在繼續聊天模型研究努力中可能會在某個時候遇到它們。

In [4]:
%%time
from langchain.document_loaders import UnstructuredFileLoader
from langchain.document_loaders import ArxivLoader

## Loading in the file

## Unstructured File Loader: Good for arbitrary "probably good enough" loader
# documents = UnstructuredFileLoader("llama2_paper.pdf").load()

## More specialized loader, won't work for everything, but simple API and usually better results
documents = ArxivLoader(query="2404.16130").load()  ## GraphRAG
# documents = ArxivLoader(query="2404.03622").load()  ## Visualization-of-Thought
# documents = ArxivLoader(query="2404.19756").load()  ## KAN: Kolmogorov-Arnold Networks
# documents = ArxivLoader(query="2404.07143").load()  ## Infini-Attention
# documents = ArxivLoader(query="2210.03629").load()  ## ReAct

CPU times: user 436 ms, sys: 85 ms, total: 521 ms
Wall time: 11.2 s


<br>


我們可以從我們的匯入中看到，這個連接器(Connector)為我們提供了兩個不同組件的存取：

-   `page_content` 是文件的實際主體，採用某種人類可解釋的格式。

-   `metadata` 是連接器(Connector)透過其資料來源提供的關於文件的相關資訊。

下面，我們可以查看文件主體的長度以了解裡面有什麼，並且可能會注意到一個難以處理的文件長度：

In [5]:
## Printing out a sample of the content
print("Number of Documents Retrieved:", len(documents))
print(f"Sample of Document 1 Content (Total Length: {len(documents[0].page_content)}):")
print(documents[0].page_content[:1000])

Number of Documents Retrieved: 1
Sample of Document 1 Content (Total Length: 89593):
From Local to Global: A GraphRAG Approach to
Query-Focused Summarization
Darren Edge1†
Ha Trinh1†
Newman Cheng2
Joshua Bradley2
Alex Chao3
Apurva Mody3
Steven Truitt2
Dasha Metropolitansky1
Robert Osazuwa Ness1
Jonathan Larson1
1Microsoft Research
2Microsoft Strategic Missions and Technologies
3Microsoft Office of the CTO
{daedge,trinhha,newmancheng,joshbradley,achao,moapurva,
steventruitt,dasham,robertness,jolarso}@microsoft.com
†These authors contributed equally to this work
Abstract
The use of retrieval-augmented generation (RAG) to retrieve relevant informa-
tion from an external knowledge source enables large language models (LLMs)
to answer questions over private and/or previously unseen document collections.
However, RAG fails on global questions directed at an entire text corpus, such
as “What are the main themes in the dataset?”, since this is inherently a query-
focused summarization (QFS) ta

<br>




相比之下，中介資料(Metadata)將更保守地調整大小，以至於成為您喜愛的聊天模型的可行脈絡資訊(Context)組件：

In [6]:
pprint(documents[0].metadata)

{
    'Published': '2025-02-19',
    'Title': 'From Local to Global: A Graph RAG Approach to Query-Focused Summarization',
    'Authors': 'Darren Edge, Ha Trinh, Newman Cheng, Joshua Bradley, Alex Chao, Apurva Mody, Steven Truitt, Dasha 
Metropolitansky, Robert Osazuwa Ness, Jonathan Larson',
    'Summary': 'The use of retrieval-augmented generation (RAG) to retrieve relevant\ninformation from an external 
knowledge source enables large language models\n(LLMs) to answer questions over private and/or previously unseen 
document\ncollections. However, RAG fails on global questions directed at an entire text\ncorpus, such as "What are
the main themes in the dataset?", since this is\ninherently a query-focused summarization (QFS) task, rather than 
an explicit\nretrieval task. Prior QFS methods, meanwhile, do not scale to the quantities of\ntext indexed by 
typical RAG systems. To combine the strengths of these\ncontrasting methods, we propose GraphRAG, a graph-based 
approach to question\nanswering over private text corpora that scales with both the generality of\nuser questions 
and the quantity of source text. Our approach uses an LLM to\nbuild a graph index in two stages: first, to derive 
an entity knowledge graph\nfrom the source documents, then to pregenerate community summaries for all\ngroups of 
closely related entities. Given a question, each community summary is\nused to generate a partial response, before 
all partial responses are again\nsummarized in a final response to the user. For a class of global 
sensemaking\nquestions over datasets in the 1 million token range, we show that GraphRAG\nleads to substantial 
improvements over a conventional RAG baseline for both the\ncomprehensiveness and diversity of generated answers.'
}

<br>

雖然接受中介資料(Metadata)格式並完全忽略主體可能很誘人，但有一些關鍵的選擇功能無法在不深入完整文字(Text)的情況下實現：

-   **中介資料(Metadata)不被保證。** 在 `arxiv` 的情況下，論文摘要、標題、作者和日期是提交的必要組件，所以能夠查詢它們並不令人驚訝。但對於任意 PDF 或網頁，情況不一定如此。

-   **Agent 將無法更深入地了解文件內容。** 摘要很好知道並可以按原樣使用，但不提供與主體在任何容量上互動的直接路徑（至少從我們學到的內容來看不是這樣）。

-   **Agent 仍然無法同時對太多文件進行分析推理(Reasoning)。** 也許在 MRKL/ReAct 範例中，您可以將這兩個摘要合併為一個脈絡資訊(Context)並詢問一些問題。但是當您需要同時與 5 個文件互動時會發生什麼？整個目錄呢？很快，您會注意到您的脈絡資訊(Context)視窗將被資訊過載，僅僅是為了總結或甚至列出您感興趣的文件！


<br>

## **第三部分：** 轉換文件


一旦文件被載入，如果我們打算將它們作為脈絡資訊(Context)傳遞到我們的 LLM 中，它們通常需要被轉換。一種轉換方法被稱為**分塊(chunking)**，它將大塊內容分解為較小的片段。這種技術很有價值，因為它有助於[最佳化從向量資料庫返回內容的相關性](https://www.pinecone.io/learn/chunking-strategies/)。

LangChain 提供[各種文件轉換器](https://python.langchain.com/docs/integrations/document_transformers/)，其中我們將使用 [`RecursiveCharacterTextSplitter`](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter)。這個選項將允許我們基於自然停止點的偏好來分割我們的文件。

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " ", ""],
)

## Some nice custom preprocessing
# documents[0].page_content = documents[0].page_content.replace(". .", "")
docs_split = text_splitter.split_documents(documents)

# def include_doc(doc):
#     ## Some chunks will be overburdened with useless numerical data, so we'll filter it out
#     string = doc.page_content
#     if len([l for l in string if l.isalpha()]) < (len(string)//2):
#         return False
#     return True

# docs_split = [doc for doc in docs_split if include_doc(doc)]
print(len(docs_split))

83


In [8]:
for i in (0, 1, 2, 15, -1):
    pprint(f"[Document {i}]")
    print(docs_split[i].page_content)
    pprint("="*64)

[Document 0]

From Local to Global: A GraphRAG Approach to
Query-Focused Summarization
Darren Edge1†
Ha Trinh1†
Newman Cheng2
Joshua Bradley2
Alex Chao3
Apurva Mody3
Steven Truitt2
Dasha Metropolitansky1
Robert Osazuwa Ness1
Jonathan Larson1
1Microsoft Research
2Microsoft Strategic Missions and Technologies
3Microsoft Office of the CTO
{daedge,trinhha,newmancheng,joshbradley,achao,moapurva,
steventruitt,dasham,robertness,jolarso}@microsoft.com
†These authors contributed equally to this work
Abstract
The use of retrieval-augmented generation (RAG) to retrieve relevant informa-
tion from an external knowledge source enables large language models (LLMs)
to answer questions over private and/or previously unseen document collections.
However, RAG fails on global questions directed at an entire text corpus, such
as “What are the main themes in the dataset?”, since this is inherently a query-
focused summarization (QFS) task, rather than an explicit retrieval task. Prior
QFS methods, meanwhile, do not scal

================================================================

[Document 1]

propose GraphRAG, a graph-based approach to question answering over private
text corpora that scales with both the generality of user questions and the quantity
of source text. Our approach uses an LLM to build a graph index in two stages:
first, to derive an entity knowledge graph from the source documents, then to pre-
generate community summaries for all groups of closely related entities. Given a
question, each community summary is used to generate a partial response, before
all partial responses are again summarized in a final response to the user. For a
class of global sensemaking questions over datasets in the 1 million token range,
we show that GraphRAG leads to substantial improvements over a conventional
RAG baseline for both the comprehensiveness and diversity of generated answers.
1
Introduction
Retrieval augmented generation (RAG) (Lewis et al., 2020) is an established approach to using
LLMs to answer queries based on data that is too large to contain in a language model’s

...

from few-shot exemplars specialized to those domains.
The LLM can also be prompted to extract claims about detected entities. Claims are important
factual statements about entities, such as dates, events, and interactions with other entities. As
with entities and relationships, in-context learning exemplars can provide domain-specific guidance.
Claim descriptions extracted from the example tetx chunk are as follows:
• NeoChip’s shares surged during their first week of trading on the NewTech Exchange.
• NeoChip debuted as a publicly listed company on the NewTech Exchange.
• Quantum Systems acquired NeoChip in 2016 and held ownership until NeoChip went pub-
lic.
See Appendix A for prompts and details on our implementation of entity and claim extraction.
3.1.3
Entities & Relationships →Knowledge Graph
The use of an LLM to extract entities, relationships, and claims is a form of abstractive summariza-
tion – these are meaningful summaries of concepts that, in the case of relationships and 

================================================================

[Document -1]

54.8
-3.68
0.003
C1
TS
47.92
52.08
-2.41
0.126
46.64
53.36
-2.91
0.04
C2
TS
48.8
51.2
-2.23
0.179
48.32
51.68
-2.12
0.179
C3
TS
48.08
51.92
-2.23
0.179
48.32
51.68
-2.56
0.074
C0
SS
35.12
64.88
-6.17
<0.001
41.44
58.56
-4.82
<0.001
C1
SS
40.32
59.68
-4.83
<0.001
45.2
54.8
-3.19
0.017
C2
SS
40.4
59.6
-4.67
<0.001
44.88
55.12
-3.65
0.003
C3
SS
40.48
59.52
-4.69
<0.001
45.6
54.4
-2.86
0.043
TS
SS
43.6
56.4
-3.96
<0.001
46
54
-2.68
0.066
C0
C1
46.96
53.04
-2.87
0.037
47.6
52.4
-2.17
0.179
C0
C2
48.4
51.6
-2.06
0.197
48.48
51.52
-1.61
0.321
C1
C2
49.84
50.16
-1
0.952
49.28
50.72
-1.6
0.321
C0
C3
48.4
51.6
-1.8
0.29
47.2
52.8
-2.62
0.071
C1
C3
49.76
50.24
0
1
48.8
51.2
-1.29
0.321
C2
C3
50
50
0
1
48.8
51.2
-1.84
0.262
26


================================================================

<br>


我們的分塊(Chunking)方法相當最單純的(naive)，但突出了為我們的應用程式至少讓某些東西運作的容易性。我們努力保持分塊大小較小，以便我們的模型能夠有效地將其作為脈絡資訊(Context)使用，但我們如何對所有這些片段進行分析推理(Reasoning)？

**當為任意文件集擴展和最佳化這種方法時，一些潛在選項包括：**

-   識別邏輯中斷或綜合結果(Synthesis)技術（手動、自動、LLM 輔助等）。
-   旨在構建富含獨特和相關資訊的分塊，避免冗餘以最大化資料庫實用性。
-   自訂分塊以適應文件的性質，確保分塊在脈絡資訊(Context)上相關且連貫。
-   在每個分塊中包含關鍵概念、關鍵字或中介資料(Metadata)片段，以改善資料庫中的可搜尋性和相關性。
-   持續評估分塊效果，並準備調整策略以在大小和內容豐富性之間達到最佳平衡。
-   考慮階層結構(Hierarchy)系統（隱式生成或明確指定）以改善檢索(Retrieval)嘗試。
    -   如果感興趣，請查看 [**LlamaIndex 索引指南中的樹狀結構**](https://docs.llamaindex.ai/en/stable/module_guides/indexing/index_guide.html#tree-index) 作為起點。



<br>

## **第四部分：[練習]** 精煉(Refinement)摘要


為了自動對大型文件進行分析推理(Reasoning)，一個潛在的想法可能是使用 LLM 創建密集摘要或知識庫。類似於我們在上一個 notebook 中透過槽填充維護對話的運行歷史紀錄(History)，保持整個文件的運行歷史紀錄(History)有什麼問題嗎？

在這一節中，我們專注於 LLM 的一個令人興奮的應用：**自動精煉(Refinement)、強制和大量整合(Aggregate)資料**。具體來說，我們將實作一個簡單但有用的可運行物件(Runnable)，它使用 while 迴圈和 Running State Chain 制定來總結一組文件分塊。這個過程通常被稱為 [**「文件精煉(document refinement)」**](https://js.langchain.com/v0.1/docs/modules/chains/document/refine/)，很大程度上類似於我們之前以對話為中心的槽填充練習；唯一的區別是現在我們處理的是大型文件而不是不斷增長的聊天歷史紀錄(History)。


<!-- > <img src="https://drive.google.com/uc?export=view&id=1J2XR8Cc8YSkVJMiJCknMkgA02mBT8riZ" width=1000px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/doc_refine.png" width=1000px/>
>
> From [**Refine | LangChain**🦜️🔗](https://js.langchain.com/v0.1/docs/modules/chains/document/refine/)

<br>

#### **DocumentSummaryBase 模型**


就像上一個 notebook 中的 `KnowledgeBase` 類別一樣，我們可以創建一個 `DocumentSummaryBase` 結構，設計用於封裝文件的精髓。下面的一個將使用 `running_summary` 欄位來查詢模型以獲得最終摘要，同時嘗試使用 `main_ideas` 和 `loose_ends` 欄位作為瓶頸，以防止運行摘要移動得太快。這是我們必須透過提示(Prompt)工程強制執行的東西，所以也提供了 `summary_prompt`，它顯示了這些資訊將如何使用。請根據需要隨意修改它，以使其適用於您選擇的模型。




In [9]:
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticOutputParser

from langchain_nvidia_ai_endpoints import ChatNVIDIA

from pydantic import BaseModel, Field
from typing import List
from IPython.display import clear_output


class DocumentSummaryBase(BaseModel):
    running_summary: str = Field("", description="Running description of the document. Do not override; only update!")
    main_ideas: List[str] = Field([], description="Most important information from the document (max 3)")
    loose_ends: List[str] = Field([], description="Open questions that would be good to incorporate into summary, but that are yet unknown (max 3)")


summary_prompt = ChatPromptTemplate.from_template(
    "You are generating a running summary of the document. Make it readable by a technical user."
    " After this, the old knowledge base will be replaced by the new one. Make sure a reader can still understand everything."
    " Keep it short, but as dense and useful as possible! The information should flow from chunk to (loose ends or main ideas) to running_summary."
    " The updated knowledge base keep all of the information from running_summary here: {info_base}."
    "\n\n{format_instructions}. Follow the format precisely, including quotations and commas"
    "\n\nWithout losing any of the info, update the knowledge base with the following: {input}"
)

<br>



我們也將利用這個機會從上一個 notebook 中帶回 `RExtract` 函式(function)：

In [10]:
def RExtract(pydantic_class, llm, prompt):
    '''
    Runnable Extraction module
    Returns a knowledge dictionary populated by slot-filling extraction
    '''
    parser = PydanticOutputParser(pydantic_object=pydantic_class)
    instruct_merge = RunnableAssign({'format_instructions' : lambda x: parser.get_format_instructions()})
    def preparse(string):
        if '{' not in string: string = '{' + string
        if '}' not in string: string = string + '}'
        string = (string
            .replace("\\_", "_")
            .replace("\n", " ")
            .replace("\]", "]")
            .replace("\[", "[")
        )
        # print(string)  ## Good for diagnostics
        return string
    return instruct_merge | prompt | llm | preparse | parser


<br>


考慮到這一點，以下程式碼在 for 迴圈中invoke Running State Chain 來疊代(Iterate)您的文件！唯一必要的修改應該是 `parse_chain` 實作，它應該透過來自上一個 notebook 的適當配置的 `RExtract` 鏈(Chain)傳遞狀態。在此之後，系統應該運作得相當好，以維護文件的運行摘要（儘管根據使用的模型，可能需要對提示(Prompt)進行一些調整）。

In [11]:
latest_summary = ""

## TODO: Use the techniques from the previous notebook to complete the exercise
def RSummarizer(knowledge, llm, prompt, verbose=False):
    '''
    Exercise: Create a chain that summarizes
    '''
    ###########################################################################################
    ## START TODO:

    def summarize_docs(docs):        
        ## TODO: Initialize the parse_chain appropriately; should include an RExtract instance.
        ## HINT: You can get a class using the <object>.__class__ attribute...
        # parse_chain = RunnableAssign({'info_base' : (lambda x: None)})
        parse_chain = RunnableAssign({'info_base' : RExtract(knowledge.__class__, llm, prompt)})
        ## TODO: Initialize a valid starting state. Should be similar to notebook 4
        # state = {}
        state = {'info_base' : knowledge}

        global latest_summary  ## If your loop crashes, you can check out the latest_summary
        
        for i, doc in enumerate(docs):
            ## TODO: Update the state as appropriate using your parse_chain component
            state['input'] = doc.page_content
            state = parse_chain.invoke(state)

            assert 'info_base' in state 
            if verbose:
                print(f"Considered {i+1} documents")
                pprint(state['info_base'])
                latest_summary = state['info_base']
                clear_output(wait=True)

        return state['info_base']
        
    ## END TODO
    ###########################################################################################
    
    return RunnableLambda(summarize_docs)

# instruct_model = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1").bind(max_tokens=4096)
instruct_model = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1").bind(max_tokens=4096)
instruct_llm = instruct_model | StrOutputParser()

## Take the first 10 document chunks and accumulate a DocumentSummaryBase
summarizer = RSummarizer(DocumentSummaryBase(), instruct_llm, summary_prompt, verbose=True)
summary = summarizer.invoke(docs_split[:15])

Considered 15 documents


DocumentSummaryBase(
    running_summary='GraphRAG, a graph-based approach for question answering, utilizes an LLM over private text 
corpora. It outperforms conventional RAG in global sensemaking queries when using GPT-4 as the LLM. The LLM is 
prompted to extract entities, relationships, and short descriptions from text chunks. NeoChip, a low-power 
processor firm for wearables and IoT devices, was previously owned by Quantum Systems until it became publicly 
traded in 2016. The LLM can extract domain-specific entities and relationships based on few-shot exemplars for 
in-context learning.',
    main_ideas=[
        'GraphRAG, a graph-based approach, uses LLM to build a graph index for question answering.',
        'GraphRAG outperforms conventional RAG for global sensemaking queries using GPT-4 as the LLM.',
        'The LLM extracts entities, relationships, and short descriptions from text chunks, which can be tailored 
using few-shot exemplars for in-context learning.'
    ],
    loose_ends=[
        'How does GraphRAG significantly improve over conventional RAG for global sensemaking questions?',
        'What types of global sensemaking questions does GraphRAG aim to answer?',
        'What is the impact of the context window size on the performance of GraphRAG?'
    ]
)

In [12]:
pprint(latest_summary)

DocumentSummaryBase(
    running_summary='GraphRAG, a graph-based approach for question answering, utilizes an LLM over private text 
corpora. It outperforms conventional RAG in global sensemaking queries when using GPT-4 as the LLM. The LLM is 
prompted to extract entities, relationships, and short descriptions from text chunks. NeoChip, a low-power 
processor firm for wearables and IoT devices, was previously owned by Quantum Systems until it became publicly 
traded in 2016. The LLM can extract domain-specific entities and relationships based on few-shot exemplars for 
in-context learning.',
    main_ideas=[
        'GraphRAG, a graph-based approach, uses LLM to build a graph index for question answering.',
        'GraphRAG outperforms conventional RAG for global sensemaking queries using GPT-4 as the LLM.',
        'The LLM extracts entities, relationships, and short descriptions from text chunks, which can be tailored 
using few-shot exemplars for in-context learning.'
    ],
    loose_ends=[
        'How does GraphRAG significantly improve over conventional RAG for global sensemaking questions?',
        'What types of global sensemaking questions does GraphRAG aim to answer?',
        'What is the impact of the context window size on the performance of GraphRAG?'
    ]
)



<br>

## **第五部分：** 綜合結果(Synthesis)資料處理

當我們結束使用 LLM 進行文件摘要的探索時，重要的是要承認更廣泛的脈絡資訊(Context)和潛在挑戰。雖然我們已經展示了提取簡潔、有意義摘要的可行方法，但讓我們考慮為什麼這樣的方法至關重要以及它所涉及的複雜性。

**精煉(Refinement)的一般化(Generalized)**


重要的是要注意，這種「漸進式摘要」技術僅僅是一個起始鏈(Chain)，它對初始資料和所需輸出格式做出很少假設。相同的技術可以廣泛擴展，以生成具有已知中介資料(Metadata)、積極假設和下游目標的綜合結果(Synthesis)精煉(Refinement)。

**考慮這些潛在應用：**

1.  **整合(Aggregate)資料**：構建將原始資料從文件分塊轉換為連貫、有用摘要的結構。

2.  **分類和子主題分析**：創建將分塊中的見解分類到定義類別中的系統，追蹤每個類別中出現的子主題。

3.  **整合(Aggregate)為密集資訊分塊**：將這些結構精煉(Refinement)為將見解提煉成緊湊片段，並富含直接引用以進行更深入分析。

這些應用暗示了創建**特定領域知識圖譜(Knowledge Graph)**，可以被對話聊天模型存取和遍訪(Traverse)。一些實用工具已經存在，可以透過 [**LangChain 知識圖譜(Knowledge Graphs)**](https://python.langchain.com/docs/tutorials/graph/) 等工具自動生成這些。雖然您可能需要開發階層結構(Hierarchy)和工具來構建和遍訪(Traverse)這樣的結構，但當您可以為您的使用案例適當精煉(Refinement)足夠的知識圖譜(Knowledge Graph)時，這是一個可行的選項！對於那些對更高級的知識圖譜(Knowledge Graph)構建技術感興趣的人，這些技術依賴於更大的系統和向量相似性，我們發現 [**LangChain x Neo4j 文章**](https://blog.langchain.dev/using-a-knowledge-graph-to-implement-a-devops-rag-application/) 很有趣。

**解決大規模資料處理的挑戰**


雖然我們的方法開啟了令人興奮的可能性，但它並非沒有挑戰，特別是在處理大量資料時：

-   **通用預處理限制**：雖然摘要相對直接，但開發在各種脈絡資訊(Context)中普遍有效的階層結構(Hierarchy)是具有挑戰性的。

-   **粒度和訪問(Navigate)成本**：在階層結構(Hierarchy)中實現詳細粒度可能是資源密集的，需要複雜的整合(Aggregate)或廣泛的分支(Branch)來維持每次互動的可管理脈絡資訊(Context)大小。

-   **對精確指令執行的依賴**：使用我們目前的工具訪問(Navigate)這樣的階層結構(Hierarchy)將嚴重依賴於具有強大提示(Prompt)工程的強大指令調整模型。推論(Inference)延遲和參數預測錯誤的風險可能很大，所以使用 LLM 進行此操作可能是一個挑戰。

當您在課程中進步時，請追蹤這些挑戰如何透過後續技術得到解決。



<br>

## **第六部分：** 總結

這個 notebook 的目標是介紹圍繞聊天模型大型文件處理的問題和技術。在下一個 notebook 中，我們將調查一個具有非常不同優缺點集合的補充工具；**使用內嵌模型的語義檢索(Retrieval)**。



### <font color="#76b900">**做得很好！**</font>

### **下一步：**


1.  **[可選]** 重新訪問(Navigate) notebook 頂部的**「值得思考的問題」部分**，並思考一些可能的答案。

2.  **[可選]** 這個 notebook 包括一些基本的文件處理鏈(Chain)，但沒有涉及 [Map Reduce](https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain/) 鏈(Chain)，這些也非常有用並建立在大致相同的直覺上。這些是一個很好的下一步，所以請查看它們！



<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>
